In [1]:
from get_dataset import * 
import numpy as np
from tensorflow.keras import models, layers
import os
import json
import copy

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-22 19:41:46.467228: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
BACH_SIZE = 64
MODELS_FOLDER = "saved_models"
MAX_EPOCHS = 30
REPEAT_COUNT = 15

In [3]:
ds_train = get_dataset("train")
ds_test = get_dataset("test")

2023-04-22 19:41:48.240933: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7529 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [4]:
def remap_labels_monolit(x, y):
    if y == 10:
        return x, tf.convert_to_tensor([0]*11, dtype=tf.float32)
    if y == 11:
        y = 10
    return x, tf.one_hot(y, 11)

In [5]:
ds_train_monolit = ds_train.map(remap_labels_monolit).batch(BACH_SIZE)
ds_test_monolit = ds_test.map(remap_labels_monolit).batch(BACH_SIZE)

In [6]:
def save_json(obj):
    f = open(MODELS_FOLDER+"/history3.json","w")
    json.dump(obj, f)
    f.close()
    
full_history = {}
full_history["monolit"] = {}
full_history["seperate1"] = {}
full_history["seperate2"] = {}

In [7]:
def get_model(output_size):
    return models.Sequential([
            layers.Input(shape=[124, 129]),
            layers.Bidirectional(layers.GRU(128)),
            layers.Dense(output_size)
        ])

In [8]:
for iter in range(REPEAT_COUNT):
    m = get_model(11)
    m.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )

    m.fit(
        ds_train_monolit,
        validation_data=ds_test_monolit,
        epochs=MAX_EPOCHS,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
            tf.keras.callbacks.ModelCheckpoint(
                filepath=MODELS_FOLDER+"/monolit_"+str(iter),
                save_weights_only=True,
                monitor='val_accuracy',
                mode='max',
                save_best_only=True)])

    full_history["monolit"][iter] = m.history.history
    save_json(full_history)

Epoch 1/30


2023-04-22 19:41:48.719102: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-22 19:41:48.719954: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-22 19:41:48.720831: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

    348/Unknown - 6s 9ms/step - loss: 1.3421 - accuracy: 0.4437

2023-04-22 19:41:55.402608: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-22 19:41:55.555852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-22 19:41:55.556717: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

348/348 [==============================] - 7s 11ms/step - loss: 1.3421 - accuracy: 0.4437 - val_loss: 0.9107 - val_accuracy: 0.6446
Epoch 2/30
348/348 [==============================] - 4s 10ms/step - loss: 0.7158 - accuracy: 0.7033 - val_loss: 0.7159 - val_accuracy: 0.7118
Epoch 3/30
348/348 [==============================] - 4s 10ms/step - loss: 0.6264 - accuracy: 0.7280 - val_loss: 0.6656 - val_accuracy: 0.7199
Epoch 4/30
348/348 [==============================] - 4s 10ms/step - loss: 0.6547 - accuracy: 0.7163 - val_loss: 0.7192 - val_accuracy: 0.6998
Epoch 5/30
348/348 [==============================] - 4s 10ms/step - loss: 0.5631 - accuracy: 0.7498 - val_loss: 0.6098 - val_accuracy: 0.7514
Epoch 6/30
348/348 [==============================] - 3s 9ms/step - loss: 0.5770 - accuracy: 0.7476 - val_loss: 0.6761 - val_accuracy: 0.7277
Epoch 7/30
348/348 [==============================] - 4s 10ms/step - loss: 0.4976 - accuracy: 0.7703 - val_loss: 0.6004 - val_accuracy: 0.7332
Epoch 8/30


...

348/348 [==============================] - 6s 11ms/step - loss: 1.3303 - accuracy: 0.4340 - val_loss: 1.0040 - val_accuracy: 0.5910
Epoch 2/30
348/348 [==============================] - 4s 9ms/step - loss: 0.7487 - accuracy: 0.6931 - val_loss: 0.8183 - val_accuracy: 0.6803
Epoch 3/30
348/348 [==============================] - 3s 9ms/step - loss: 0.8473 - accuracy: 0.6637 - val_loss: 0.9410 - val_accuracy: 0.6241
Epoch 4/30
348/348 [==============================] - 4s 9ms/step - loss: 0.7177 - accuracy: 0.7103 - val_loss: 0.7811 - val_accuracy: 0.6884
Epoch 5/30
348/348 [==============================] - 4s 10ms/step - loss: 0.5891 - accuracy: 0.7444 - val_loss: 0.6296 - val_accuracy: 0.7465
Epoch 6/30
348/348 [==============================] - 3s 9ms/step - loss: 0.5869 - accuracy: 0.7471 - val_loss: 0.7261 - val_accuracy: 0.7280
Epoch 7/30
348/348 [==============================] - 4s 9ms/step - loss: 0.7949 - accuracy: 0.6839 - val_loss: 0.9055 - val_accuracy: 0.6576
Epoch 8/30
348/

2023-04-22 19:49:29.100563: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-22 19:49:29.101714: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-22 19:49:29.102454: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

    346/Unknown - 5s 9ms/step - loss: 1.3180 - accuracy: 0.4450

2023-04-22 19:49:34.606082: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-22 19:49:34.607104: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-22 19:49:34.607819: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

348/348 [==============================] - 6s 10ms/step - loss: 1.3144 - accuracy: 0.4464 - val_loss: 0.8217 - val_accuracy: 0.6774
Epoch 2/30
348/348 [==============================] - 4s 9ms/step - loss: 0.6654 - accuracy: 0.7236 - val_loss: 0.6966 - val_accuracy: 0.7267
Epoch 3/30
348/348 [==============================] - 3s 9ms/step - loss: 0.5552 - accuracy: 0.7594 - val_loss: 0.6427 - val_accuracy: 0.7429
Epoch 4/30
348/348 [==============================] - 3s 9ms/step - loss: 0.5833 - accuracy: 0.7478 - val_loss: 0.5818 - val_accuracy: 0.7559
Epoch 5/30
348/348 [==============================] - 4s 9ms/step - loss: 0.5688 - accuracy: 0.7528 - val_loss: 0.7495 - val_accuracy: 0.6994
Epoch 6/30
348/348 [==============================] - 3s 9ms/step - loss: 0.5943 - accuracy: 0.7394 - val_loss: 0.6935 - val_accuracy: 0.7248
Epoch 7/30
348/348 [==============================] - 3s 9ms/step - loss: 0.5758 - accuracy: 0.7471 - val_loss: 0.8238 - val_accuracy: 0.6592
Epoch 7: early s

In [9]:
def remap_silence_rest(x, y):
    if y == 12:
        return x, 1
    else:
        return x, 0

ds = ds_train.map(remap_silence_rest)
ds_train_separete_1 = tf.data.Dataset.sample_from_datasets(
    [ds.filter(lambda x, y: y == 0),
     ds.filter(lambda x, y: y == 1)], [0.5, 0.5])
ds_train_separete_1 = ds_train_separete_1.batch(BACH_SIZE)

ds = ds_test.map(remap_silence_rest)
ds_test_separete_1 = tf.data.Dataset.sample_from_datasets(
    [ds.filter(lambda x, y: y == 0),
     ds.filter(lambda x, y: y == 1)], [0.5, 0.5])
ds_test_separete_1 = ds_test_separete_1.batch(BACH_SIZE)

In [10]:
def remap_labels_rest(x, y):
    if y == 10:
        return x, tf.convert_to_tensor([0]*10, dtype=tf.float32)
    return x, tf.one_hot(y, 10)

def filter_silence(x, y):
    return y != 11

ds_train_seperate_2 = ds_train.filter(filter_silence).map(remap_labels_rest).batch(BACH_SIZE)
ds_test_seperate_2 = ds_test.filter(filter_silence).map(remap_labels_rest).batch(BACH_SIZE)

In [11]:
for iter in range(REPEAT_COUNT):
    m = get_model(2)
    m.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )

    m.fit(
        ds_train_separete_1,
        validation_data=ds_test_separete_1,
        epochs=MAX_EPOCHS,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
            tf.keras.callbacks.ModelCheckpoint(
                filepath=MODELS_FOLDER+"/seperate1_"+str(iter),
                save_weights_only=True,
                monitor='val_accuracy',
                mode='max',
                save_best_only=True)])

    full_history["seperate1"][iter] = m.history.history
    save_json(full_history)

Epoch 1/30


2023-04-22 19:49:56.403425: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-22 19:49:56.404429: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-22 19:49:56.405183: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

    348/Unknown - 6s 10ms/step - loss: 0.0161 - accuracy: 0.9967

2023-04-22 19:50:02.943916: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype float and shape [1,2]
	 [[{{node Placeholder/_6}}]]
2023-04-22 19:50:02.944264: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_18' with dtype int32
	 [[{{node Placeholder/_18}}]]
2023-04-22 19:50:03.117023: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	

348/348 [==============================] - 7s 13ms/step - loss: 0.0161 - accuracy: 0.9967 - val_loss: 1.1804e-05 - val_accuracy: 1.0000
Epoch 2/30
348/348 [==============================] - 5s 11ms/step - loss: 9.6496e-06 - accuracy: 1.0000 - val_loss: 7.2039e-06 - val_accuracy: 1.0000
Epoch 3/30
348/348 [==============================] - 4s 10ms/step - loss: 4.9703e-06 - accuracy: 1.0000 - val_loss: 5.4036e-06 - val_accuracy: 1.0000
Epoch 4/30
348/348 [==============================] - 5s 10ms/step - loss: 3.0106e-06 - accuracy: 1.0000 - val_loss: 4.4099e-06 - val_accuracy: 1.0000
Epoch 5/30
348/348 [==============================] - 5s 11ms/step - loss: 2.1383e-06 - accuracy: 1.0000 - val_loss: 3.8349e-06 - val_accuracy: 1.0000
Epoch 6/30
348/348 [==============================] - 5s 11ms/step - loss: 1.6036e-06 - accuracy: 1.0000 - val_loss: 3.4098e-06 - val_accuracy: 1.0000
Epoch 7/30
348/348 [==============================] - 5s 10ms/step - loss: 1.2156e-06 - accuracy: 1.0000 - va

...

348/348 [==============================] - 7s 11ms/step - loss: 0.0182 - accuracy: 0.9971 - val_loss: 7.5644e-05 - val_accuracy: 1.0000
Epoch 2/30
348/348 [==============================] - 5s 10ms/step - loss: 3.5395e-05 - accuracy: 1.0000 - val_loss: 1.6853e-05 - val_accuracy: 1.0000
Epoch 3/30
348/348 [==============================] - 5s 10ms/step - loss: 8.5815e-06 - accuracy: 1.0000 - val_loss: 8.2440e-06 - val_accuracy: 1.0000
Epoch 4/30
348/348 [==============================] - 5s 11ms/step - loss: 4.9026e-06 - accuracy: 1.0000 - val_loss: 4.8534e-06 - val_accuracy: 1.0000
Epoch 5/30
348/348 [==============================] - 5s 11ms/step - loss: 3.2853e-06 - accuracy: 1.0000 - val_loss: 3.3184e-06 - val_accuracy: 1.0000
Epoch 6/30
348/348 [==============================] - 5s 11ms/step - loss: 2.3007e-06 - accuracy: 1.0000 - val_loss: 2.3836e-06 - val_accuracy: 1.0000
Epoch 7/30
348/348 [==============================] - 5s 10ms/step - loss: 1.6548e-06 - accuracy: 1.0000 - va

2023-04-22 20:23:18.252665: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-22 20:23:18.253888: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-22 20:23:18.254611: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

    348/Unknown - 6s 9ms/step - loss: 0.0206 - accuracy: 0.9942

2023-04-22 20:23:24.814120: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-22 20:23:24.815087: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-22 20:23:24.815832: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

348/348 [==============================] - 7s 11ms/step - loss: 0.0206 - accuracy: 0.9942 - val_loss: 3.7705e-05 - val_accuracy: 1.0000
Epoch 2/30
348/348 [==============================] - 5s 10ms/step - loss: 2.4579e-05 - accuracy: 1.0000 - val_loss: 1.7841e-05 - val_accuracy: 1.0000
Epoch 3/30
348/348 [==============================] - 5s 11ms/step - loss: 1.2107e-05 - accuracy: 1.0000 - val_loss: 1.2374e-05 - val_accuracy: 1.0000
Epoch 4/30
348/348 [==============================] - 5s 10ms/step - loss: 7.8730e-06 - accuracy: 1.0000 - val_loss: 9.5304e-06 - val_accuracy: 1.0000
Epoch 5/30
348/348 [==============================] - 5s 10ms/step - loss: 5.5949e-06 - accuracy: 1.0000 - val_loss: 7.3269e-06 - val_accuracy: 1.0000
Epoch 6/30
348/348 [==============================] - 5s 11ms/step - loss: 4.1391e-06 - accuracy: 1.0000 - val_loss: 5.9093e-06 - val_accuracy: 1.0000
Epoch 7/30
348/348 [==============================] - 5s 11ms/step - loss: 3.1071e-06 - accuracy: 1.0000 - va

In [12]:
for iter in range(REPEAT_COUNT):
    m = get_model(10)
    m.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )

    m.fit(
        ds_train_seperate_2,
        validation_data=ds_test_seperate_2,
        epochs=MAX_EPOCHS,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
            tf.keras.callbacks.ModelCheckpoint(
                filepath=MODELS_FOLDER+"/seperate2_"+str(iter),
                save_weights_only=True,
                monitor='val_accuracy',
                mode='max',
                save_best_only=True)])

    full_history["seperate2"][iter] = m.history.history
    save_json(full_history)
    

Epoch 1/30


2023-04-22 20:25:44.078485: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-22 20:25:44.079460: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-22 20:25:44.080177: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

    317/Unknown - 5s 9ms/step - loss: 1.3446 - accuracy: 0.4235

2023-04-22 20:25:49.578910: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-22 20:25:49.715029: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-22 20:25:49.715913: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

319/319 [==============================] - 6s 11ms/step - loss: 1.3415 - accuracy: 0.4252 - val_loss: 0.9311 - val_accuracy: 0.6225
Epoch 2/30
319/319 [==============================] - 3s 10ms/step - loss: 0.7065 - accuracy: 0.6984 - val_loss: 0.6746 - val_accuracy: 0.7167
Epoch 3/30
319/319 [==============================] - 3s 10ms/step - loss: 0.5639 - accuracy: 0.7463 - val_loss: 0.6460 - val_accuracy: 0.7213
Epoch 4/30
319/319 [==============================] - 3s 10ms/step - loss: 0.5139 - accuracy: 0.7626 - val_loss: 0.6214 - val_accuracy: 0.7365
Epoch 5/30
319/319 [==============================] - 3s 10ms/step - loss: 0.4926 - accuracy: 0.7722 - val_loss: 0.6635 - val_accuracy: 0.7135
Epoch 6/30
319/319 [==============================] - 3s 10ms/step - loss: 0.4717 - accuracy: 0.7772 - val_loss: 0.5229 - val_accuracy: 0.7599
Epoch 7/30
319/319 [==============================] - 3s 10ms/step - loss: 0.4618 - accuracy: 0.7771 - val_loss: 0.8605 - val_accuracy: 0.6558
Epoch 8/30

...

Epoch 1/30


2023-04-22 20:34:31.229568: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-04-22 20:34:31.253629: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-22 20:34:31.254404: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

    318/Unknown - 5s 9ms/step - loss: 1.4394 - accuracy: 0.3868

2023-04-22 20:34:36.528036: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-22 20:34:36.529012: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-22 20:34:36.529779: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

319/319 [==============================] - 6s 11ms/step - loss: 1.4375 - accuracy: 0.3878 - val_loss: 1.0330 - val_accuracy: 0.5914
Epoch 2/30
319/319 [==============================] - 3s 10ms/step - loss: 0.7091 - accuracy: 0.6997 - val_loss: 0.7633 - val_accuracy: 0.6912
Epoch 3/30
319/319 [==============================] - 3s 10ms/step - loss: 0.5668 - accuracy: 0.7488 - val_loss: 0.6510 - val_accuracy: 0.7192
Epoch 4/30
319/319 [==============================] - 3s 10ms/step - loss: 0.5840 - accuracy: 0.7416 - val_loss: 0.6692 - val_accuracy: 0.7132
Epoch 5/30
319/319 [==============================] - 3s 10ms/step - loss: 0.5365 - accuracy: 0.7592 - val_loss: 0.6389 - val_accuracy: 0.7256
Epoch 6/30
319/319 [==============================] - 3s 10ms/step - loss: 0.5039 - accuracy: 0.7703 - val_loss: 0.6029 - val_accuracy: 0.7542
Epoch 7/30
319/319 [==============================] - 3s 10ms/step - loss: 0.5630 - accuracy: 0.7429 - val_loss: 0.9228 - val_accuracy: 0.6331
Epoch 8/30